<a href="https://colab.research.google.com/github/silentfortin/ai-portfolio/blob/main/01-contactease-python/ContactEase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ContactEase Solutions – Contact Manager in Python

> 👨‍💻 Developed as part of the **AI Engineering Master – Week 1**

This project is a simple command-line contact manager application written in Python using Object-Oriented Programming (OOP). It allows users to add, edit, remove, view, and search contacts, and it stores contact data in a JSON file for persistence between sessions.

## 🔧 Features
- Add, show, edit, delete, and search contacts
- Save and load contacts from a JSON file
- Command-line interface with color-coded feedback
- Input validation and user-friendly prompts

🔗 GitHub Repository:
[📁 ai-portfolio](https://github.com/silentfortin/ai-portfolio/)


In [4]:
# Import necessary modules
import uuid
import json

from json.decoder import JSONDecodeError
from typing import List, Optional

In [5]:
# ANSI escape sequences for colored terminal output
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
CYAN = "\033[96m"
RESET = "\033[0m"

In [6]:
class Contact:
    """Represents a contact with name, surname, phone number, and a unique ID.

    Args:
        name (str): First name of the contact.
        surname (str): Last name of the contact.
        phone_number (str): Phone number.
        id (Optional[str]): Optional UUID string. Auto-generated if not provided.

    Example:
        contact = Contact("Alice", "Rossi", "123456789")
    """

    def __init__(self, name: str, surname: str, phone_number: str, id: Optional[str] = None) -> None:
        self.id: str = id if id else str(uuid.uuid4())
        self.name: str = name
        self.surname: str = surname
        self.phone_number: str = phone_number

    def __str__(self) -> str:
        """Returns a string representation of the contact.

        Returns:
            str: Formatted contact string with ANSI colors.
        """
        return f'{YELLOW}Full name: {self.name} {self.surname} | Phone: {self.phone_number}{RESET}'


class ContactsBook:
    """Manages a collection of Contact objects with CRUD and file operations."""

    def __init__(self) -> None:
        self.contacts: List[Contact] = []

    def add_contact(self, contact: Contact) -> None:
        """Adds a contact to the list and saves to file.

        Args:
            contact (Contact): The contact to add.

        Returns:
            None
        """
        self.contacts.append(contact)
        self.save_to_file()
        print(f'{GREEN}{contact.name} {contact.surname} has been added.{RESET}\n')

    def show_all_contacts(self) -> None:
        """Displays all contacts in the book.

        Returns:
            None
        """
        if self.contacts:
            print(f'\n{CYAN}Your contacts:{RESET}')
            for contact in self.contacts:
                print(contact)
        else:
            print(f'{YELLOW}Your contacts book is empty.{RESET}')

    def edit_contact(self, contact: Contact, mod: str, updated_var: str) -> None:
        """Edits a field of a contact.

        Args:
            contact (Contact): Contact to edit.
            mod (str): Field to edit ('n', 's', or 'p').
            updated_var (str): New value.

        Returns:
            None
        """
        match mod:
            case 'n':
                contact.name = updated_var
                print(f'{GREEN}Contact name modified.{RESET}')
            case 's':
                contact.surname = updated_var
                print(f'{GREEN}Contact surname modified.{RESET}')
            case 'p':
                contact.phone_number = updated_var
                print(f'{GREEN}Contact phone number modified.{RESET}')
            case _:
                print(f'{RED}ERROR: Contact not modified.{RESET}')
                return
        self.save_to_file()

    def remove_contact(self, contact_to_remove: Contact) -> None:
        """Removes a contact from the list.

        Args:
            contact_to_remove (Contact): Contact to remove.

        Returns:
            None
        """
        for contact in self.contacts:
            if contact.id == contact_to_remove.id:
                self.contacts.remove(contact)
                self.save_to_file()
                print(f'{YELLOW}Contact {contact.name} {contact.surname} removed.{RESET}')
                return
        print('Contact not found.')

    def search_contact(self, name: str, surname: str) -> Optional[Contact]:
        """Searches a contact by full name (case-insensitive).

        Args:
            name (str): First name.
            surname (str): Last name.

        Returns:
            Optional[Contact]: Found contact or None.
        """
        name = name.strip().lower()
        surname = surname.strip().lower()

        for contact in self.contacts:
            if contact.name.lower() == name and contact.surname.lower() == surname:
                return contact
        return None

    def save_to_file(self, filename: str = 'contacts_book.json') -> None:
        """Saves all contacts to a JSON file.

        Args:
            filename (str): Path to JSON file.

        Returns:
            None
        """
        try:
            with open(filename, 'w') as f:
                json.dump([vars(c) for c in self.contacts], f, indent=2)
        except JSONDecodeError as e:
            print(f'Error encoding JSON: {e}')
        except Exception as e:
            print(f'Unexpected error saving file: {e}')

    def load_from_file(self, filename: str = 'contacts_book.json') -> None:
        """Loads contacts from a JSON file.

        Args:
            filename (str): Path to JSON file.

        Returns:
            None
        """
        try:
            with open(filename, 'r') as file:
                data = json.load(file)
                self.contacts = [Contact(**c) for c in data]
        except FileNotFoundError:
            print(f'File "{filename}" not found. Starting fresh.')
            self.contacts = []
        except JSONDecodeError as e:
            print(f'Invalid JSON content: {e}')
            self.contacts = []
        except Exception as e:
            print(f'Unexpected error loading file: {e}')
            self.contacts = []

In [7]:
# Helper function(s)

def is_valid_phone(phone: str) -> bool:
    """Checks if a phone number is valid.

    Args:
        phone (str): Phone number string.

    Returns:
        bool: True if valid.
    """
    return phone.isdigit() and 3 <= len(phone) <= 15

def prompt_valid_phone() -> str:
    """Prompts user until a valid phone number is entered.

    Returns:
        str: Validated phone number.
    """
    while True:
        phone = input('Insert phone number: ').strip()
        if is_valid_phone(phone):
            return phone
        print(f'{YELLOW}Invalid phone number. Only digits, 3–15 digits allowed.{RESET}')

def prompt_non_empty_input(field_name: str) -> str:
    """Prompts user for a non-empty value.

    Args:
        field_name (str): The name of the field (e.g., 'name').

    Returns:
        str: User input.
    """
    while True:
        value = input(f'Insert contact {field_name}: ').strip()
        if value:
            return value
        print(f'{RED}{field_name.capitalize()} cannot be empty.{RESET}')


def add_contact_helper(book: ContactsBook) -> None:
    """Handles the user interaction for adding a new contact.

    Args:
        book (ContactsBook): The contacts book instance.

    Returns:
        None
    """
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')
    phone = prompt_valid_phone()

    new_contact = Contact(name, surname, phone)
    check_duplicate = book.search_contact(new_contact.name, new_contact.surname)

    if check_duplicate is None:
        book.add_contact(new_contact)
    else:
        print(f'{YELLOW}The contact {new_contact.name} {new_contact.surname} already exists. Do you want to add it anyways?{RESET}')
        add_choice = input('Select [Y] or [X]: ').strip()
        if add_choice.lower() == 'y':
            book.add_contact(new_contact)
        else:
            print(f'{YELLOW}The contact has not been added.\n{RESET}')

def edit_contact_helper(book: ContactsBook) -> None:
    """Handles user interaction to edit a contact's name, surname, or phone.

    Args:
        book (ContactsBook): The contacts book instance.

    Returns:
        None
    """
    possible_choices = ['n', 'p', 's']

    print(f'{YELLOW}Insert the name and the surname of the contact that you want to edit.{RESET}')
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')

    contact_to_edit = book.search_contact(name, surname)

    if not contact_to_edit:
        print(f'{RED}Contact not found.{RESET}')
        return

    while True:
        edit_choice = input('\nPress [N] to edit the name\nPress [S] to edit the surname\nPress [P] to edit the phone number: ').strip().lower()
        if edit_choice in possible_choices:
            break
        else:
            print(f'{YELLOW}Invalid value. Please enter N, S, or P.{RESET}')

    new_value = prompt_valid_phone() if edit_choice == 'p' else prompt_non_empty_input('new value')
    book.edit_contact(contact_to_edit, edit_choice, new_value)

def remove_contact_helper(book: ContactsBook) -> None:
    """Handles the user interaction for removing a contact by name and surname.

    Args:
        book (ContactsBook): The contacts book instance.

    Returns:
        None
    """
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')

    contact_to_remove = book.search_contact(name, surname)
    if not contact_to_remove:
        print(f'{RED}Contact not found.{RESET}')
        return

    book.remove_contact(contact_to_remove)

def search_contact_helper(book: ContactsBook) -> None:
    """Searches for a contact by name and surname and prints the result.

    Args:
        book (ContactsBook): The contacts book instance.

    Returns:
        None
    """
    name = prompt_non_empty_input('name')
    surname = prompt_non_empty_input('surname')

    contact = book.search_contact(name, surname)
    if contact:
        print(contact)
    else:
        print(f'{RED}Contact not found.{RESET}')

### CLI

In [8]:
book = ContactsBook()
# Load existing contacts or initialize empty list if file not found
book.load_from_file()


def main_menu() -> None:
    """
    Displays the main CLI menu and routes the user's choice
    to the appropriate contact management function.

    Returns:
        None
    """

    while True:
        print(f'\n{CYAN}=== Contact Book ==={RESET}')
        print(f'{GREEN}1. Add contact{RESET}')
        print(f'{GREEN}2. Show contacts{RESET}')
        print(f'{GREEN}3. Modify contact{RESET}')
        print(f'{GREEN}4. Remove contact{RESET}')
        print(f'{GREEN}5. Search contact{RESET}')
        print(f'{RED}6. Exit{RESET}')

        choice = input('Choose an option (1–6): ').strip()

        if choice == '1':
            add_contact_helper(book)
        elif choice == '2':
            book.show_all_contacts()
        elif choice == '3':
            edit_contact_helper(book)
        elif choice == '4':
            remove_contact_helper(book)
        elif choice == '5':
            search_contact_helper(book)
        elif choice == '6':
            print(f'{RED}Exiting...{RESET}')
            break
        else:
            print(f'{RED}Invalid choice. Try again.{RESET}')


if __name__ == '__main__':
    main_menu()


File "contacts_book.json" not found. Starting fresh.

=== Contact Book ===
1. Add contact
2. Show contacts
3. Modify contact
4. Remove contact
5. Search contact
6. Exit
Choose an option (1–6): 1
Insert contact name: a
Insert contact surname: b
Insert phone number: 123
a b has been added.


=== Contact Book ===
1. Add contact
2. Show contacts
3. Modify contact
4. Remove contact
5. Search contact
6. Exit
Choose an option (1–6): 1
Insert contact name: c
Insert contact surname: d
Insert phone number: 123
c d has been added.


=== Contact Book ===
1. Add contact
2. Show contacts
3. Modify contact
4. Remove contact
5. Search contact
6. Exit
Choose an option (1–6): 
Invalid choice. Try again.

=== Contact Book ===
1. Add contact
2. Show contacts
3. Modify contact
4. Remove contact
5. Search contact
6. Exit
Choose an option (1–6): 2

Your contacts:
Full name: a b | Phone: 123
Full name: c d | Phone: 123

=== Contact Book ===
1. Add contact
2. Show contacts
3. Modify contact
4. Remove contact
5

---

## 📚 References & Resources

- [Try and Except in Python](https://pythonbasics.org/try-except/)

- [Python User Input: Handling, Validation, and Best Practices](https://www.datacamp.com/tutorial/python-user-input)

- [Python Match Case Statement](https://www.geeksforgeeks.org/python/python-match-case-statement/)

- [5 Efficient Ways to Convert a Class Object to a Dictionary in Python](https://blog.finxter.com/5-efficient-ways-to-convert-a-class-object-to-a-dictionary-in-python/)

- [Dictionary Unpacking](https://pieriantraining.com/python-unpack-dictionary-a-comprehensive-guide/)

- [Working With JSON Data in Python](https://realpython.com/python-json/#write-a-json-file-with-python)

- [Stack Overflow – How do I print colored text to the terminal?](https://stackoverflow.com/questions/287871/how-do-i-print-colored-text-to-the-terminal)

- [ANSI command line colors with Python](https://python.code-maven.com/ansi-command-line-colors-with-python)

- ChatGPT – used for improving documentation clarity and polishing markdown sections


